<div >
<img src = "figs/bannerMLMkting.png" />
</div>

# Sistemas de Recomendación. Recomendación basada en ítems


### Estructura de los sistemas de recomendación

Los sistemas de recomendación constan principalmente de 2 componentes:

   - Generación de candidatos: En esta  etapa, el sistema parte de un corpus potencialmente enorme y genera un subconjunto mucho más pequeño de candidatos a recomendar. Por ejemplo, el generador de recomendaciones de YouTube reduce miles de millones de videos cargados en su plataforma a cientos o miles de potenciales candidatos.
   
   - Ranking o Puntuación: En esta etapa se elige un modelo que clasifica a los candidatos para seleccionar el conjunto de candidatos que va a mostrar al usuario.
   
   
El orden de estas etapas no sigue un orden establecido y pueden intercambiarse y hasta fusionarse. También es importante mencionar que algunos sistemas de recomendación incorporan una etapa de reclasificación donde se suman restricciones adicionales antes de mostrar los candidatos finales. Por ejemplo, se eliminan candidatos que al usuario no le gustaron explícitamente o se aumenta la puntuación de contenido más nuevo. La reclasificación también se suele utilizar para garantizar que los contenidos sugeridos sean diversos, nuevos y/o populares.


### Recomendadores basados en conocimiento

Los recomendadores basados en  conocimiento utilizan principalmente artículos que rara vez se usan o compran. Si los artículos son raramente comprados es muy difícil tener información previa a partir de la cuál de ellos se puede generar una recomendación. La venta de bienes raíces es un ejemplo de esto. La compra de una casa suele ser a menudo una compra de una vez en la vida, no se tiene por lo tanto información adicional para generar recomendaciones.

En casos como este, el sistema de recomendación funciona similar a lo que haría un ser humano. Le pregunta al usuario sobre preferencias y detalles; y a partir de estos, genera recomendaciones. Volviendo al ejemplo anterior, el sistema podría preguntar sobre qué tipo de propiedad prefiere, qué barrios, cuál es su presupuesto, etc.; y a partir de estos ofrecer una recomendación.

### Filtrado colaborativo:

El filtrado colaborativo aprovecha el poder de la colaboración para generar recomendaciones. Los filtros colaborativos son uno de los sistemas de recomendación más populares utilizados en la industria y han tenido un gran éxito para empresas como Amazon. En términos generales, el filtrado colaborativo puede clasificarse en dos tipos:

1. **Filtrado basado en ítems**: La idea de este tipo de filtrado es que si un grupo de personas ha calificado dos elementos de manera similar, entonces los dos elementos deben ser similares.

Por lo tanto, si a una persona le gusta un artículo en particular, es probable que también esté interesada en el otro artículo. Como se puede ver en la figura siguiente esta es otra de las estrategias que utiliza Amazon: recomienda productos a partir del historial de navegación y las compras, y con esta información genera sugerencias:

<div  style="max-width: 80%;">
<img src = "figs/item_jeans.png"/>
</div>

2. **Filtrado basado en usuarios**: La idea principal detrás del filtrado basado en el usuario es que si somos capaces de encontrar usuarios que compraron artículos similares y les gustaron en el pasado, es más probable que también compren artículos similares en el futuro.

<div  style="max-width: 80%;">
<img src = "figs/aguacate.png"/>
</div>

Por lo tanto, estos modelos recomiendan artículos a un usuario que a otros usuarios similares también le han gustado. Amazon, por ejemplo, usa este tipo de método para generar recomendaciones. En la figura a continuación nos aparece la frase "Los clientes que compraron este producto también compraron" y a continuación aparecen las sugerencias:

<div  style="max-width: 80%;">
<img src = "figs/usuario_libros.png"/>
</div>

### Filtrado basado en contenido


A diferencia de los filtros colaborativos, los sistemas basados en contenido no requieren datos relacionados a otros individuos u actividades pasadas.  Por el contrario estos, brindan recomendaciones basadas en el perfil del usuario y los metadatos que se tiene sobre elementos particulares:


<div  style="max-width: 40%;">
<img src = "figs/Homero_movies.png"/>
</div>




Netflix es un buen ejemplo de este tipo de sistemas como lo muestra la figura a continuación. Basado en nuestro perfil, cómo calificamos la película y metadatos de la película nos genera sugerencias.


<div  style="max-width: 40%;">
<img src = "figs/netflix.png"/>
</div>


En este caso habrá menos diversidad en las recomendaciones, ya que por ejemplo si sólo usamos este tipo de sistemas puede ser que potencialmente nos gusten dramas británicos, pero nunca lo sabremos, a menos que decidamos probarlo de forma autónoma.


## Primera aproximación en `Python`

Implementaremos dos sistemas de recomendación: uno simple y otro basado en conocimiento. Estos ilustrarán de manera sencilla como funcionan los sistemas en la práctica y nos permitirán tener una mejor base para cuando busquemos aboradar sistemas más complejos en *cuadernos* futuros.

###  Implementaciones en  `Python`

Para ilustrar el funcionamiento de estos recomendadores simples vamos a utilizar una versión simplificada y traducida de la base de datos de películas: [MovieLens](https://grouplens.org/datasets/movielens/latest/) provista abiertamente por [grouplens](https://grouplens.org/about/what-is-grouplens/) para: **"avanzar la teoría y la práctica de la computación social mediante la construcción y la comprensión de sistemas *(de recomendación)* utilizados por personas reales".**

Carguemos entonces las librerías y los datos:


In [10]:
import gdown

# Replace 'file_id' with the actual file ID
file_id = '1rro8KcZWb44H8bcrRJp2iNOpZBRLSYLS'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file (optional: specify output file name)
gdown.download(url, 'metadata_limpia.csv', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1rro8KcZWb44H8bcrRJp2iNOpZBRLSYLS
To: /content/metadata_limpia.csv
100%|██████████| 3.39M/3.39M [00:00<00:00, 206MB/s]


'metadata_limpia.csv'

In [11]:
# Cargamos las librerías a utilizar
import pandas as pd
import numpy as np

# Cargamos y visualizamos las primeras filas de los datos
df = pd.read_csv("metadata_limpia.csv")
df.head()

,Unnamed: 0,titulo,duracion,voto_promedio,numero_votos,ano,genero
0,1,Toy Story,81,7.7,5415,1995,animacion
1,2,Toy Story,81,7.7,5415,1995,comedia
2,3,Jumanji,104,6.9,2413,1995,adventura
3,4,Jumanji,104,6.9,2413,1995,fantasia
4,5,Grumpier Old Men,101,6.5,92,1995,romance


La base contiene el título, la duración (en minutos), el voto promedio y el número de votos que obtuvo, el año de estreno, y el género. Note que si una película tiene más de un género aparece repetida. Por ejemplo, Toy Story, aparece bajo el género de animación y de comedia.

Para esta aplicación omitiremos el género y por lo tanto nos quedaremos con sólo una entrada por película (los invito a que prueben si los resultados cambian o no al no omitir las entradas repetidas):

### Un  recomendador basado en conocimiento

Construyamos entonces ahora un recomendador que incorpore preferencias de los usuarios. Para ello seguimos utilizando la bsase anterior y lo que el recomendadro hará es:

1. Preguntar al usuario por el género de película que está buscando.
2. Preguntar al usuario por la duración.
3. Preguntar al usuario por el rango de año de estreno de la película.
4. Con esta información recomendará la película con el $Score$ más alto.

Veamos entonces cuáles son los géneros disponibles:

In [12]:
df['genero'].value_counts()

,count
genero,
drama,15909
comedia,10752
suspenso,6718
accion,5634
romance,5003
terror,4098
documental,3659
adventura,2727
ciencia ficcion,2612


y los años disponibles:

In [13]:
df['ano'].value_counts()

,count
ano,
2015,3244
2014,3236
2013,2994
2009,2976
2011,2861
2010,2705
2016,2669
2008,2666
2012,2663


Construyamos entonces una función que nos pregunte por nuestras preferencias y que nos genere las recomendaciones. En otras palabras, estos son los tres pasos principales de la función:

   1. Obtener información del usuario sobre sus preferencias.
   2. Filtrar las películas que coincidan con las condiciones establecidas por el usuario.
   3. Calcular los valores de m y C.
   4. Retornar las sugerencias.

Llamaremos a la función `recomendador` y tendrá 2 parámetros: la base de datos sobre la que construye la recomendaciones y el percentil ($m$). Utilizaremos por ahora el mismo $m=.95$ del ejemplo anterior:

In [14]:
def recomendador(db, percentile=0.95):
    # Preguntar por los géneros preferidos
    print("Hola este es tu sistema de recomendación")
    print("Te voy a hacer unas preguntas para generar sugerencias")
    print("¿Qué género de película querés ver?")
    genre = input()

    # Preguntar por la duración mínima de la película
    print("¿Cuál es la duración mínima que querés? (en minutos)")
    low_time = int(input())

    # Preguntar por la duración máxima de la película
    print("¿Cuál es la duración máxima que querés? (en minutos)")
    high_time = int(input())

        # Preguntar por el rango de los años en que la película fue estrenada
    print("Ahora te voy a preguntar por el rango de años de estreno de la película")

    print("¿Cuál es el año de estreno más viejo que querés ver? ")
    low_year = int(input())

    print("¿Cuál es el año de estreno más nuevo que querés ver? ")
    high_year = int(input())


    # Definimos una nueva variable que va a guardar las películas preferidas, para ello copiamos los contenidos de la bas
    movies = db.copy()

    # Filtramos basado en las respuestas
    movies = movies[(movies['genero'] == genre) &
                    (movies['duracion'] >= low_time) &
                    (movies['duracion'] <= high_time) &
                    (movies['ano'] >= low_year) &
                    (movies['ano'] <= high_year)]

    # Calculamos los valores de C y m
    C = movies['voto_promedio'].mean()
    m = movies['numero_votos'].quantile(percentile)


    # Consideramos sólo películas que tienen más de m votos, y las guardamos en un data.frame
    q_movies = movies.copy().loc[movies['numero_votos'] >= m]

    # Calculamos el score usando la  formula de IMDB
    q_movies['score'] = q_movies.apply(lambda x: (x['numero_votos']/(x['numero_votos']+m) * x['voto_promedio'])
                                       + (m/(m+x['numero_votos']) * C)
                                       ,axis=1)

    # Ordenamos las películas en orden descendiente de su score
    q_movies = q_movies.sort_values('score', ascending=False)

    return q_movies

Utilicemos entonces esta función para generar recomendaciones de películas de drama de entre media y 2 horas de duración que fueron estrenadas entre 1990 y 2005:

In [31]:
recomendador(df).head()

Hola este es tu sistema de recomendación
Te voy a hacer unas preguntas para generar sugerencias
¿Qué género de película querés ver?
accion
¿Cuál es la duración mínima que querés? (en minutos)
120
¿Cuál es la duración máxima que querés? (en minutos)
180
Ahora te voy a preguntar por el rango de años de estreno de la película
¿Cuál es el año de estreno más viejo que querés ver? 
1965
¿Cuál es el año de estreno más nuevo que querés ver? 
1990


,Unnamed: 0,titulo,duracion,voto_promedio,numero_votos,ano,genero,score
2025,2026,The Empire Strikes Back,124,8.2,5998,1980,accion,7.747642
526,527,Star Wars,121,8.1,6778,1977,accion,7.711177
2046,2047,Return of the Jedi,135,7.9,4763,1983,accion,7.459453
7395,7396,Scarface,170,8.0,3017,1983,accion,7.368691
2036,2037,Aliens,137,7.7,3282,1986,accion,7.217779


Vemos que las películas retornadas por la función satisfacen todas las condiciones que ingresamos. Notemos también que dado que usamos la métrica definida por IMDB, que se explicará más adelante. Note también que estas recomendaciones incluyen *American History X*, que es una de mis películas favoritas. Por mi parte, estaría muy satisfecho con estas sugerencias.

### Un recomendador simple

Para construir nuestro primer recomendador simple vamos a seguir los siguientes pasos:

   1. Decidir cuáles son los atributos deseados de la película que queremos recomendar. Esto nos va a permitir reducir el espacio de candidatos potenciales.
   2. Elegir una métrica que nos permita calificar y ordenar las películas, para luego calcularla para nuestro subconjunto.
   3. Recomendar películas basadas en esta métrica en orden decreciente de sus puntajes.


La parte crucial de este recomendador es el segundo punto. Generar una métrica que nos permita ordenar las películas. La elección de esta es arbitraria, sin embargo es deseable que esta métrica sea robusta. Por ejemplo, podríamos usar el voto promedio. Sin embargo, sabemos que la media no es una medida robusta. Para ilustrarlo, imaginemos que tenemos una película con calificación promedio de 9 por 100.000 usuarios, esta se colocará debajo de una película con una calificación promedio de 9,5 por 100 usuarios. Esto no es deseable, ya que es muy probable que una película vista y calificada sólo por 100 personas atienda a un nicho muy específico y puede que no atraiga tanto al individuo promedio como la primera. Otra de las desventajas de la media es que no es robusta a la micronumerosidad, es decir, que tenga pocos votos. A medida que el número de votos aumente, es probable que este se acerque al verdadero valor poblacional. Es decir, una película calificada con 10/10 por cinco usuarios no significa necesariamente que sea una buena película.

Por lo tanto, necesitamos una métrica que pueda sobreponerse a estas falencias, de forma tal que, por ejemplo, ordené una película de gran éxito con una calificación de 8 por 100.000 usuarios sobre una película con una calificación de 9 por 100 usuarios.

En este *cuaderno* vamos a utilizar la métrica propuesta por [IMDB](https://www.imdb.com/chart/top/?ref_=nv_mv_250) para clasificar las mejores 250 películas. Esta métrica sigue la siguiente fórmula:

$$
Score = \left( \frac{v}{v+m}\times R + \frac{m}{v+m}\times C \right)
$$


donde:

   - $v$ es el número de votos.
   - $m$ es el mínimo número de votos que tiene que haber recibido una pelicula para que figure en el ranking.
   - $R$ es el voto promedio de la película
   - $C$ es el voto promedio de *todas* la películas en la base.
    
##### Creando el recomendador

Tenemos entonces todos los ingredientes para crear nuestro recomendador. Comencemos calculando el $Score$. Notemos que tiene como parámetro $m$. En nuestra aplicación lo fijaremos en el percentil 95 (los invito a que lo cambien y prueben cómo cambian las recomendaciones).



La base contiene el título, la duración (en minutos), el voto promedio y el número de votos que obtuvo, el año de estreno, y el género. Note que si una película tiene más de un género aparece repetida. Por ejemplo, Toy Story, aparece bajo el género de animación y de comedia.

Para esta aplicación omitiremos el género y por lo tanto nos quedaremos con sólo una entrada por película (los invito a que prueben si los resultados cambian o no al no omitir las entradas repetidas):

In [16]:
df_nodup = df.drop_duplicates(subset ="titulo")
df_nodup

,Unnamed: 0,titulo,duracion,voto_promedio,numero_votos,ano,genero
0,1,Toy Story,81,7.7,5415,1995,animacion
2,3,Jumanji,104,6.9,2413,1995,adventura
4,5,Grumpier Old Men,101,6.5,92,1995,romance
6,7,Waiting to Exhale,127,6.1,34,1995,comedia
9,10,Father of the Bride Part II,106,5.7,173,1995,comedia
...,...,...,...,...,...,...,...
64928,64929,Deep Hearts,58,0.0,0,1981,documental
64933,64934,Shadow of the Blair Witch,45,7.0,2,2000,misterio
64935,64936,The Burkittsville 7,30,7.0,1,2000,terror
64936,64937,Caged Heat 3000,85,3.5,1,1995,ciencia ficcion


La columna `voto_promedio` es el promedio de votos que dieron los usuarios a la película. Este voto toma valores  entre 1 y 10, siendo 10 el voto más alto. La columna `numero_votos` es el número de votos que recibió la película. Este sirve también como una proxy de cuanta gente vió la película.

Estamos ahora en condiciones de construir nuestro primer recomendador simple.

Calculamos entonces el parámetro $m$:

In [17]:
m = df_nodup['numero_votos'].quantile(0.95)
m

590.0

Con esto podemos ver  cuáles son las que superaron el mínimo número de votos que tiene que haber recibido una película para que figure en el ranking:

In [18]:
q_movies = df_nodup[df_nodup['numero_votos'] >= m]

q_movies.head()

,Unnamed: 0,titulo,duracion,voto_promedio,numero_votos,ano,genero
0,1,Toy Story,81,7.7,5415,1995,animacion
2,3,Jumanji,104,6.9,2413,1995,adventura
10,11,Heat,170,7.7,1886,1995,accion
21,22,GoldenEye,130,6.6,1194,1995,adventura
34,35,Casino,178,7.8,1343,1995,drama


In [19]:
q_movies.shape

(1627, 7)

Vemos que nos quedamos con aproximadamente el 5%, como esperábamos. Calculemos  $C$, la calificación promedio de todas las películas en el conjunto de datos:

In [20]:
C = df_nodup['voto_promedio'].mean()
C

5.725672018207542

Con esto podemos calcular entonces nuestro $Score$ para cada película. Para ello crearemos una función:

In [21]:
def calc_score(votos, promedio, m=m, C=C):
    v = votos
    R = promedio
    # Calcular el score
    return (v/(v+m) * R) + (m/(m+v) * C)

Y aplicaremos la función, indicando que la operación es por filas (`axis=1`):

In [22]:
q_movies.loc[:,'numero_votos']

,numero_votos
0,5415
2,2413
10,1886
21,1194
34,1343
...,...
63956,598
63973,748
64009,2712
64158,1440


In [23]:
q_movies1 = q_movies.copy()
q_movies1.loc[:,'score'] = calc_score(q_movies1.loc[:,'numero_votos'],q_movies1.loc[:,'voto_promedio'])

In [24]:
# Mostramos el top 10
q_movies.head(10)

,Unnamed: 0,titulo,duracion,voto_promedio,numero_votos,ano,genero
0,1,Toy Story,81,7.7,5415,1995,animacion
2,3,Jumanji,104,6.9,2413,1995,adventura
10,11,Heat,170,7.7,1886,1995,accion
21,22,GoldenEye,130,6.6,1194,1995,adventura
34,35,Casino,178,7.8,1343,1995,drama
38,39,Ace Ventura: When Nature Calls,90,6.1,1128,1995,comedia
65,66,Twelve Monkeys,129,7.4,2470,1995,ciencia ficcion
70,71,Babe,89,6.0,756,1995,fantasia
80,81,Clueless,97,6.9,828,1995,comedia
98,99,Se7en,127,8.1,5915,1995,misterio


In [25]:
q_movies['score'] = calc_score(q_movies['numero_votos'],q_movies['voto_promedio'])

<ipython-input-25-e7e014ead425>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q_movies['score'] = calc_score(q_movies['numero_votos'],q_movies['voto_promedio'])


In [26]:
# Mostramos el top 10
q_movies.head(10)

,Unnamed: 0,titulo,duracion,voto_promedio,numero_votos,ano,genero,score
0,1,Toy Story,81,7.7,5415,1995,animacion,7.506019
2,3,Jumanji,104,6.9,2413,1995,adventura,6.669280
10,11,Heat,170,7.7,1886,1995,accion,7.229542
21,22,GoldenEye,130,6.6,1194,1995,adventura,6.310844
34,35,Casino,178,7.8,1343,1995,drama,7.166863
38,39,Ace Ventura: When Nature Calls,90,6.1,1128,1995,comedia,5.971447
65,66,Twelve Monkeys,129,7.4,2470,1995,ciencia ficcion,7.077172
70,71,Babe,89,6.0,756,1995,fantasia,5.879752
80,81,Clueless,97,6.9,828,1995,comedia,6.411387
98,99,Se7en,127,8.1,5915,1995,misterio,7.884650


El último paso para recomendar las películas es ordenarlas y mostrarlas al usuario:


In [27]:
# Ordenamos
q_movies1 = q_movies1.sort_values('score', ascending=False)

# Mostramos el top 10
q_movies1.head(10)

,Unnamed: 0,titulo,duracion,voto_promedio,numero_votos,ano,genero,score
644,645,The Shawshank Redemption,142,8.5,8358,1994,drama,8.317070
1682,1683,The Godfather,175,8.5,6024,1972,drama,8.252517
20733,20734,The Dark Knight,152,8.3,12269,2008,drama,8.181884
5158,5159,Fight Club,139,8.3,9678,1999,drama,8.152079
603,604,Pulp Fiction,154,8.3,8670,1994,suspenso,8.135977
727,728,Forrest Gump,142,8.2,8147,1994,comedia,8.032911
25272,25273,Inception,148,8.1,14075,2010,accion,8.004476
1093,1094,Schindler's List,195,8.3,4436,1993,drama,7.997801
36397,36398,Whiplash,105,8.3,4376,2014,drama,7.994150
35323,35324,Interstellar,169,8.1,11187,2014,adventura,7.981052


Notemos que nuestra recomendación señala el mismo top 3 que IMDB (*$^*$imagen tomada el 28 de Abril de 2022)*:

<div  style="max-width: 60%;">
<img src = "figs/IMDB.png"/>
</div>


En el top figura [The Shawshank Redemption](https://es.wikipedia.org/wiki/The_Shawshank_Redemption) que fue estrenada como "Sueño de fuga" en Hispanoamérica y como "Sueños de libertad" o "Escape a la libertad" en Argentina.

Nuestro simple recomendador genera recomendaciones sencillas pero no muy personalizadas. Veamos entonces otro sistema que nos permita recomendar sugerencias más personalizadas.

# Referencias

- Banik, R. (2018). Hands-on recommendation systems with Python: start building powerful and personalized, recommendation engines with Python. Packt Publishing Ltd.

- Covington, P., Adams, J., & Sargin, E. (2016). Deep Neural Networks for YouTube recommendations. Proceedings of the 10th ACM Conference on Recommender Systems. https://doi.org/10.1145/2959100.2959190

- Google developers. (n.d.). Recommendation systems. Google. Consultado en Abril 3, 2022, de https://developers.google.com/machine-learning/recommendation/overview

# Información de Sesión

In [29]:
pip install session-info

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 2.4 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8023 sha256=ac1666f267fd3a9379c64b4520b91feb946de8d4d09136288a311fe4cf2a3443
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session-info


In [30]:
import session_info

session_info.show(html=False)

-----
gdown               5.2.0
numpy               1.26.4
pandas              2.2.2
session_info        1.0.0
-----
IPython             7.34.0
jupyter_client      6.1.12
jupyter_core        5.7.2
notebook            6.5.5
-----
Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
Linux-6.1.85+-x86_64-with-glibc2.35
-----
Session information updated at 2024-11-13 23:12
